In [1]:
USE_FILE = 2            # номер используемго файла (см ниже)
PEAK_MIN_DISTANCE = 10  # минимальное расстояние между максимумами

In [2]:
# List of available data file names
# Список доступных файлов с данными
files = {
    1: "Checkme O2 _20250114084307.csv", 
    2: "Checkme O2 _20250119122308.csv", # запись в постели, я не сплю, но только что проснулся после длительного сна и еще не двигался
    3: "Checkme O2 _20250116013719.csv", # пример ночной записи с этапами сна в наморднике и без него, этапов просыпания
}

In [3]:
# Load the libraries
# Загрузка библиотек
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import find_peaks, savgol_filter

# Set the directory where the data files are located
# Установка директории, где находятся файлы с данными
DATADIR = "data"

In [4]:
# Load CSV into a pandas DataFrame
# Загрузить данные из CSV файла в датафрейм
filename = files[USE_FILE]
filename = os.path.join(DATADIR, filename)

df = pd.read_csv(
    filename,
    parse_dates=['Time'],
    date_format='%H:%M:%S %b %d %Y'
)

# Set Time as the index
# Использовать время как индекс
df.set_index('Time', inplace=True)

In [ ]:
# Напечатать содержимое датафрейма
df

In [6]:
# Remove non-numeric values from df["Oxygen Level"]
# Удалить нечисловые значения из df["Oxygen Level"] (так как там мусор кроме чисел встречается)
df["Oxygen Level"] = pd.to_numeric(df["Oxygen Level"], errors='coerce')
df.dropna(subset=['Oxygen Level'], inplace=True)

# Calculate the average oxygen level
# Вычислить средний уровень кислорода
average_O2 = df["Oxygen Level"].mean()
# Calculate the average oxygen level minus 3% - they use this threshold for apnea detection
# Вычислить средний уровень кислорода минус 3% - этот порог используется для обнаружения апноэ
average_O3_minus_3p = average_O2 * 0.97

In [ ]:
# Plot Oxygen Level vs. Time
# Построить график уровня кислорода от времени
plt.figure(figsize=(40, 4))
plt.axhline(average_O2, color='y', linestyle='--', label="Average O2")
plt.axhline(average_O3_minus_3p, color='r', linestyle='--', label="Average O2 - 3%")
plt.plot(df.index, df['Oxygen Level'])
plt.xlabel('Time')
plt.ylabel('Oxygen Level')
plt.title('Oxygen Level vs Time')
plt.grid(True)
plt.tight_layout()
plt.show()

In [8]:
# Smooth the data series
# Сгладить временной ряд
df["O2_smooth"] = savgol_filter(df["Oxygen Level"], window_length=11, polyorder=2)

In [ ]:
# Plot again
# Построить график снова
plt.figure(figsize=(40, 4))
plt.plot(df.index, df['O2_smooth'])
plt.axhline(average_O2, color='y', linestyle='--', label="Average O2")
plt.axhline(average_O3_minus_3p, color='r', linestyle='--', label="Average O2 - 3%")
plt.xlabel('Time')
plt.ylabel('O2_smooth')
plt.title('O2_smooth vs Time')
plt.grid(True)
plt.tight_layout()

In [10]:
# Use the smoothed O2 data to find peaks
# Использовать сглаженные данные O2 для поиска пиков
y = df["O2_smooth"].values

# Find_peaks returns indices in the array that are peak positions
# Find_peaks возвращает индексы в массиве, которые являются позициями пиков
peak_indices, peak_dict = find_peaks(
    y,
    distance=PEAK_MIN_DISTANCE,
)

# Convert indices back to your DataFrame index (Time)
# Преобразовать индексы обратно в индекс вашего датафрейма (Time)
peak_times = df.index[peak_indices]

# List of events, will hold (max1_idx, min_idx, max2_idx) for each event
# Список событий, будет содержать (max1_idx, min_idx, max2_idx) для каждого события
events = []

for i in range(len(peak_indices) - 1):
    # Start and end of one respiratory cycle
    # Начало и конец одного дыхательного цикла
    start_idx = peak_indices[i]
    end_idx   = peak_indices[i + 1]
    
    # Slice the O2 array in between these two peaks
    # Вырезать массив O2 между этими двумя пиками
    segment = y[start_idx : end_idx + 1]
    
    # Find the index of the min in this segment (relative to segment)
    # Найти индекс минимума в этом сегменте (относительно сегмента)
    min_local_idx = np.argmin(segment)
    
    # Convert that to an absolute index in the full data
    # Преобразовать его в абсолютный индекс в полных данных
    actual_min_idx = start_idx + min_local_idx
    
    # Store the triple: (start_of_cycle, the_minimum, next_peak)
    # Сохранить тройку: (начало_цикла, минимум, следующий_пик)
    events.append((peak_indices[i], actual_min_idx, peak_indices[i+1]))

In [11]:
# List of analysis results
# Список результатов анализа
analysis = []

for (max1, min_, max2) in events:
    # Extract the O2 values for this event
    # Извлечь значения O2 для этого события
    o2_max1 = y[max1]
    o2_min  = y[min_]
    o2_max2 = y[max2]
    
    # Extract the times for this event
    # Извлечь времена для этого события
    time_max1 = df.index[max1]
    time_min  = df.index[min_]
    time_max2 = df.index[max2]
    
    # Calculate the amplitude of O2 drop
    # Вычислить амплитуду падения O2
    drop_amp = o2_max1 - o2_min 
    # Calculate the duration of the event
    # Вычислить длительность события
    duration = (time_max2 - time_max1).total_seconds() / 60.0  # in minutes, e.g.

    # Consider the second maximum value as the baseline
    # Считать второе максимальное значение базовым
    baseline = o2_max2

    # IGNORE take an integral of the O2 values (under the baseline) in the event
    # IGNORE take into account that time between samples is not constant, but use 'Time' column to calculate the integral
    # IGNORE integral = np.trapz(y[min_:max2-1] - baseline)

    # calculate the desaturation load factor in percent-minutes
    # вычислить фактор нагрузки десатурации в процент-минутах
    integral = (baseline - o2_min) * duration
    desaturation_load = integral / baseline * 100.0
    
    # Store the analysis results
    # Сохранить результаты анализа
    analysis.append({
        "start_time": time_max1,
        "min_time": time_min,
        "end_time": time_max2,
        "start_O2": o2_max1,
        "min_O2":   o2_min,
        "end_O2":   o2_max2,
        "drop_amp": drop_amp,
        "duration": duration,
        "integral": integral,
        "desaturation_load": desaturation_load
    })

# Convert the analysis list to a DataFrame
# Преобразовать список анализа в датафрейм
analysis_df = pd.DataFrame(analysis)

In [ ]:
# Print the analysis DataFrame
# Напечатать датафрейм анализа
analysis_df

In [ ]:
# Plot the smoothed O2 data, with the peaks and minima marked
# Построить сглаженные данные O2, с отмеченными пиками и минимумами
fig, ax = plt.subplots(figsize=(40, 4))

ax.plot(df.index, y, label="O2")
plt.axhline(average_O2, color='y', linestyle='--', label="Average O2")
plt.axhline(average_O3_minus_3p, color='r', linestyle='--', label="Average O2 - 3%")

ax.plot(df.index[peak_indices], y[peak_indices], 'go', label="Detected Peaks")
minima_indices = [ev[1] for ev in events]
ax.plot(df.index[minima_indices], y[minima_indices], 'ro', label="Detected Minima")

ax.set_xlabel("Time")
ax.set_ylabel("O2 (%)")
ax.set_title("Respiratory Events (Max → Min → Max)")
ax.legend()
ax.grid(True)
plt.tight_layout()
plt.show()